In [105]:
from IPython.core.debugger import set_trace as bk
import numpy
import torch
from torch import nn
import torch.nn.functional as F
import torch.tensor as T
from transformers import ElectraModel, ElectraConfig, ElectraTokenizerFast, ElectraForMaskedLM,ElectraForPreTraining
from fastai2.text.all import *
from _utils.would_like_to_pr import *
from _utils.huggingface import *
from _utils.utils import *

In [106]:
features = numpy.load('features.npy', allow_pickle=True).item()
masked_inputs = numpy.load('masked_inputs.npy', allow_pickle=True).item()
mlm_output = numpy.load('mlm_output.npy', allow_pickle=True).item()
fake_data = numpy.load('fake_data.npy', allow_pickle=True).item()
disc_output = numpy.load('disc_output.npy', allow_pickle=True).item()

ELECTRA的code可以mask 相同位置，在製造generated的時候會把相同位置預測的id平均，在計算gen_loss的時候就等同於那個位置的多算一次

In [121]:
is_mlm_applied = torch.zeros(masked_inputs['input_ids'].shape, dtype=torch.long)
for i, poses in enumerate(masked_inputs['masked_lm_positions']):
  for j in poses:
    if j == 0: continue
    is_mlm_applied[i, j] = 1

gen_logitss = T(mlm_output['logits'])# self.generator(masked_inputs) # (B, L, vocab size)
gen_pred = gen_logitss.argmax(dim=-1)
assert torch.equal(gen_pred, T(mlm_output['preds'], dtype=torch.long))
gen_logits = torch.full([*masked_inputs['input_ids'].shape,30522], -1.)
for i, poses in enumerate(masked_inputs['masked_lm_positions']):
  for th, j in enumerate(poses):
    if j == 0: continue
    gen_logits[i, j] = gen_logitss[i, th]

# add gumbel noise and then softmax
pred_tokks = (gen_logitss + T(fake_data['noise'])).argmax(dim=-1)
assert torch.equal(pred_tokks, T(fake_data['sampled_tokens']).argmax(dim=-1))
pred_toks = torch.full(masked_inputs['input_ids'].shape, 0, dtype=torch.long)
times = torch.full(masked_inputs['input_ids'].shape, 1, dtype=torch.long)
for i, poses in enumerate(masked_inputs['masked_lm_positions']):
  for th, j in enumerate(poses):
    if j == 0: continue
    if j in poses[:th]: times[i, j] += 1
    pred_toks[i, j] += pred_tokks[i, th]
pred_toks /= times

# use predicted token to fill 15%(mlm prob) mlm applied positions
generated = ~is_mlm_applied.bool() * T(masked_inputs['input_ids'], dtype=torch.long) + is_mlm_applied * pred_toks # (B,L)
assert torch.equal(generated, T(fake_data['updated_ids'], dtype=torch.long))

# not equal to generator predicted and is at mlm applied position
labels = torch.full(masked_inputs['input_ids'].shape, -100)
mids = T(masked_inputs['masked_lm_ids']).long()
for i, poses in enumerate(masked_inputs['masked_lm_positions']):
  for th, j in enumerate(poses):
    if j == 0: continue
    labels[i, j] = mids[i, th]
is_replaced = (pred_toks != labels) * is_mlm_applied # (B, L)
assert torch.equal(is_replaced.long(), T(fake_data['is_fake_tokens']).long())

disc_logits = T(disc_output['logits']) # self.discriminator(generated) # (B, L)
disc_pred = disc_logits > 0
assert torch.equal(disc_pred.long(), T(disc_output['preds']).long())

gen_loss = CrossEntropyLossFlat()(gen_logits.float(), labels.long()) # ignore position where targ_id==-100
assert torch.allclose(gen_loss, T(mlm_output['loss']))

non_pad = generated != 0
disc_logits = disc_logits.masked_select(non_pad) # -> 1d tensor
is_replaced = is_replaced.masked_select(non_pad) # -> 1d tensor
disc_loss = nn.BCEWithLogitsLoss()(disc_logits.float(), is_replaced.float())
assert torch.allclose(disc_loss, T(disc_output['loss']))

total_loss=gen_loss * 1. + disc_loss * 50.
assert torch.equal(total_loss, T(disc_output['total_loss']))

AssertionError: 